In [2]:
#!/usr/bin/python

# This is the evaluation script for the TREC 2017 RTS evaluation
# (scenario B) with batch NIST assessor judgments, v1.0.

__author__ = 'Luchen'
import argparse
import json
import numpy
import math
import gzip
import os

qrels_path = 'Qrel/rts2017-batch-qrels.txt'
clusters_path = 'Qrel/rts2017-batch-clusters.json'
file_tweet2day = 'Qrel/rts2017-batch-tweets2dayepoch.txt'
ipath='F:/TREC2017/Analysis'
os.chdir(ipath)
path_rn='F:/TREC2017/Analysis/Result/jm/jm091117_0.1 20.0_14.0.txt'
path_rn='D:/datas/2017output/result/jm_gold1122017_0.1 21.0_10.0.txt'


#path_rn='D:/datas/2017output/result/bm25_grid2012170.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/est-res2017/bm25svr_0.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/est-res2017/bm25lR_0.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/est-res2017/bm25svr_0.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/est-res2017/bm25tunesvr_0.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/pkucst/qFB_url.txt'
path_rn='D:/datas/2017output/result/pkucst/PKUICSTRunB1'

path_rn='D:/datas/2017output/result/est-res2017/diri_linreg_est_20170.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/est-res2017/diri_svr_est_20170.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/est-res2017/jm_linreg_est_20170.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/est-res2017/jm_svr_est_20170.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/est-res2017/bm25lR_0.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/pkucst/udelRun081D-B.txt'
#path_rn='D:/datas/2017output/result/pkucst/udelRun081HT-B.txt'
#path_rn='D:/datas/2017output/result/pkucst/udelRun081HTD-B.txt'
path_rn='D:/datas/2017output/result/est-res2017/gold-diripost15217.txt'
#path_rn='D:/datas/2017output/result/dirichlet_gold_0.1 6.0_3.0.txt'

K = 10
days = []
for i in range(29, 32):
    days.append("201707%02d" % i)
for i in range(1, 6):
    days.append("201708%02d" % i)


In [3]:
# qrels dictionary, {topic: {tweetid: gain}}
qrels_dt = {}
clusters_day_dt = {}
for line in open(qrels_path).readlines():
    line = line.strip().split()
    topic = line[0]
    tweetid = line[2]
    score = int(line[3])
    if score == -2:
        score = 0
    else:
        score /= float(2)
    if topic in qrels_dt:
        qrels_dt[topic][tweetid] = score
    else:
        qrels_dt[topic] = {tweetid: score}
        clusters_day_dt[topic] = {day: [] for day in days}


# created timestamp and date for each tweetid in the qrel
# tweet2day_dt: {tweetid: date}
tweet2day_dt = {}
for line in open(file_tweet2day).readlines():
    line = line.strip().split()
    tweet2day_dt[line[0]] = line[1]

# cluster dictionaries
clusters_clusterid_dt = {}
clusters_topic_dt = json.load(open(clusters_path))
for topic in clusters_topic_dt:
    clusters_json = clusters_topic_dt[topic]["clusters"]
    if topic not in clusters_clusterid_dt:
        clusters_clusterid_dt[topic] = {}
    for clusterid in clusters_json.keys():
        for tweetid in clusters_json[clusterid]["tweets"]:
            if tweet2day_dt[tweetid] in days:
                clusters_clusterid_dt[topic][tweetid] = clusterid
                clusters_day_dt[topic][tweet2day_dt[tweetid]].append(tweetid)

In [4]:
path_rn='D:/datas/2017output/result/est-res2018/diri_linreg_est_20170.1 23.0_6.0.txt'
path_rn='D:/datas/2017output/result/est-res2018/diri_svr_est_20170.1 23.0_6.0.txt'
path_rn='D:/datas/2017output/result/est-res2018/diri_tunesvr_est_20170.1 23.0_6.0.txt'
path_rn='D:/datas/2017output/result/est-res2018/jm_linreg_est_20170.1 23.0_6.0.txt'
#path_rn='D:/datas/2017output/result/est-res2018/jm_svr_est_20170.1 23.0_6.0.txt'
path_rn='D:/datas/2017output/result/voting/estlrdjvoting.txt'
path_rn='D:/datas/2017output/result/voting/estlrbjvoting.txt'
path_rn='D:/datas/2017output/result/voting/estlrbdvoting.txt'
path_rn='D:/datas/2017output/result/voting/estsvrdjvoting.txt'
path_rn='D:/datas/2017output/result/voting/estsvrbjvoting.txt'
path_rn='D:/datas/2017output/result/voting/estsvrbdvoting.txt'
#path_rn='D:/datas/2017output/result/voting/bdbjjd.txt'
path_rn='D:/datas/2017output/result/lr-res/dirilr.txt'


In [5]:
import glob, os
count =0
#os.chdir(path_rn)

#fout=open("grid/rdata_new.txt","a+")
#fout1=open("grid/result.txt","a+")

#for file in glob.glob("*.txt"):
run_path=path_rn



# run dictionaries
# run_dt: {topic: {date: [tweetids}}
runname = ''
run_dt = {}
run_epoch_dt = {}
for line in open(run_path, "r").readlines():

    line = line.strip().split()

    runname = line[6]
    topic = line[1]
    if topic in qrels_dt:
        if topic not in run_dt:
            run_dt[topic] = {}
        day = line[0]
        if day not in run_dt[topic]:
            run_dt[topic][day] = []
        tweetid = line[3]
        run_dt[topic][day].append(tweetid)

print("{0}\t{1:5s}\t{2:6s}\t{3:6s}\t{4:6s}".format("runtag".ljust(len(runname)), "topic", "nDCGp", "nDCG1",'nDCG0'))

total_ndcg1, total_ndcgp, total_ndcg0 = 0.0, 0.0,0.0
for topic in sorted(qrels_dt):
    topic_ndcg1, topic_ndcgp,topic_ndcg0 = 0.0, 0.0,0.0
    exist_clusterids = set()
    for day in days:
        interesting = False
        max_gain_dt = {}
        tweets_fromprotocol = clusters_day_dt[topic][day]
        for tweetid in tweets_fromprotocol:
            clusterid = clusters_clusterid_dt[topic][tweetid]
            if clusterid not in exist_clusterids:
                interesting = True
                if clusterid not in max_gain_dt:
                    max_gain_dt[clusterid] = qrels_dt[topic][tweetid]
                else:
                    max_gain_dt[clusterid] = max(max_gain_dt[clusterid], qrels_dt[topic][tweetid])
        if interesting:
            if topic in run_dt and day in run_dt[topic]:
                ndcg = 0.0
                gains = []
                for tweetid in run_dt[topic][day]:
                    gain = 0.0
                    if tweetid in clusters_day_dt[topic][day]:
                        clusterid = clusters_clusterid_dt[topic][tweetid]
                        if clusterid not in exist_clusterids:
                            exist_clusterids.add(clusterid)
                            gain = qrels_dt[topic][tweetid]
                            # tweet gain is with respect to its own gain, not max gain in the cluster
                            # if clusterid in max_gain_dt:
                            #    gain = max_gain_dt[clusterid]
                    gains.append(gain)
                rank_cut = min(len(gains), K)
                dcg = 0.0
                for i in range(rank_cut):
                    gain = gains[i]
                    dcg += float(pow(2, gain) - 1) / math.log(i + 2, 2)

                # compute idcg
                top_gains = list(max_gain_dt.values())
                top_gains.sort(reverse=True)
                rank_cut = min(len(top_gains), K)
                idcg = 0.0
                top_gains = top_gains[:rank_cut]
                for i in range(rank_cut):
                    gain = top_gains[i]
                    idcg += float(pow(2, gain) - 1) / math.log(i + 2, 2)
                if idcg != 0:
                    ndcg = dcg / idcg
                topic_ndcg1 += ndcg
                topic_ndcgp += ndcg
                topic_ndcg0+=ndcg
        else:
            if topic not in run_dt or day not in run_dt[topic]:
                topic_ndcg1 += 1
                topic_ndcgp += 1
                
            elif topic in run_dt and day in run_dt[topic]:
                push_num = min(K, len(run_dt[topic][day]))
                topic_ndcgp += (1 - push_num/float(K))



    topic_ndcg1 /= len(days)
    topic_ndcgp /= len(days)
    topic_ndcg0 /= len(days)

    print("{0}\t{1:5s}\t{2:.4f}\t{3:.4f} \t{4:.4f} ".format(runname, topic, topic_ndcgp, topic_ndcg1,topic_ndcg0))
    #print len(run_path)
    #if (len(run_path)== 26) :
     #   #lamda=float(run_path[37]+ run_path[38]+ run_path[39]+run_path[40])
        #maxth=int(run_path[13:15])
        #minth=int(run_path[18:20])

        #wrstr = topic + " "+ str(round(topic_ndcgp,4)) + " " + str(round(topic_ndcg1,4)) + ' ' +  str(minth)+ ' ' +str(maxth) + '\n'
        #fout.write(wrstr)
        #print wrstr
    total_ndcg1 += topic_ndcg1
    total_ndcgp += topic_ndcgp
    total_ndcg0 += topic_ndcg0

total_ndcg1 /= len(qrels_dt)
total_ndcgp /= len(qrels_dt)
total_ndcg0 /= len(qrels_dt)

print("{0}\t{1:5s}\t{2:.4f}\t{3:.4f}\t{4:.4f}".format(runname, "All", total_ndcgp, total_ndcg1,total_ndcg0))
#tempstr = run_path + " " + str(round(total_ndcgp,4)) + " " + str(round(total_ndcg1,4)) +"\n"
#fout1.write(tempstr)
#fout1.close()
#fout.close()


runtag    	topic	nDCGp 	nDCG1 	nDCG0 
IRLAB-LDRP	RTS101	0.5120	0.4495 	0.4495 
IRLAB-LDRP	RTS104	0.0000	0.0000 	0.0000 
IRLAB-LDRP	RTS105	0.4942	0.4942 	0.3692 
IRLAB-LDRP	RTS106	0.2432	0.2432 	0.2432 
IRLAB-LDRP	RTS107	1.0000	1.0000 	0.0000 
IRLAB-LDRP	RTS111	0.3207	0.3207 	0.3207 
IRLAB-LDRP	RTS113	0.1130	0.1130 	0.1130 
IRLAB-LDRP	RTS114	0.0972	0.0972 	0.0972 
IRLAB-LDRP	RTS115	0.4778	0.4778 	0.4778 
IRLAB-LDRP	RTS116	0.0710	0.0710 	0.0710 
IRLAB-LDRP	RTS118	0.2664	0.2664 	0.2664 
IRLAB-LDRP	RTS119	0.7207	0.7207 	0.5957 
IRLAB-LDRP	RTS120	0.5320	0.5320 	0.5320 
IRLAB-LDRP	RTS123	0.0231	0.0231 	0.0231 
IRLAB-LDRP	RTS125	0.2750	0.2500 	0.1250 
IRLAB-LDRP	RTS126	0.2110	0.2110 	0.2110 
IRLAB-LDRP	RTS127	0.0000	0.0000 	0.0000 
IRLAB-LDRP	RTS130	0.1382	0.1382 	0.1382 
IRLAB-LDRP	RTS132	0.1820	0.1820 	0.1820 
IRLAB-LDRP	RTS136	0.1250	0.1250 	0.0000 
IRLAB-LDRP	RTS138	0.3585	0.3335 	0.0835 
IRLAB-LDRP	RTS139	0.1358	0.1358 	0.1358 
IRLAB-LDRP	RTS144	0.1369	0.1369 	0.1369 
IRLAB-LDRP	RTS147	0

In [98]:
len(qrels_dt)
topic=[]
for q in qrels_dt:
    topic.append(q)
len(topic)

97

In [68]:
os.chdir(ipath)
ip=[]
with open('tid95-2.txt','r') as fin:
    line= fin.readlines()
    for l in line:
        l=l.strip().split(' ')
        #tpn=l[0]
        tpid=l[1]
        ip.append(tpid)      
print len(ip)
s=list(set(topic)-set(ip))
print s

95
['RTS192', 'RTS107']


In [ ]:
import glob
import os
os.chdir(path_rn)
for file in glob.glob("*.txt"):
    run_path=file
    print (run_path,len(run_path),run_path[18:20])